In [1]:
# Computer Vision and Pattern Recognition (Section-A)
# Final Assignment CNN 
# MD MUSTAFIZUR RAHMAN (ID: 17-34272-1)

import numpy as np                   
import matplotlib.pyplot as plt      
import random
                        
# MNIST dataset is included in Keras
from keras.datasets import mnist     

from keras.models import Sequential  

#layers to be used in our model 
from keras.layers.core import Dense, Dropout, Activation 
# NumPy tools in keras
from keras.utils import np_utils                         

# import some additional tools

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
from keras.layers.normalization import BatchNormalization

In [2]:
# Reload the MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
#adding an additional dimension to represent the single-channel
X_train = X_train.reshape(60000, 28, 28, 1) 
X_test = X_test.reshape(10000, 28, 28, 1)

# change integers to 32-bit floating point numbers
X_train = X_train.astype('float32')         
X_test = X_test.astype('float32')

# normalizing each value for each pixel for the entire vector for each input
X_train /= 255                              
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Training matrix shape (60000, 28, 28, 1)
Testing matrix shape (10000, 28, 28, 1)


In [4]:
# number of unique digits
nb_classes = 10 

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [5]:

model = Sequential()                                 

# CNN Layer 1
model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))  # 32 different kernel size=3 X 3
model.add(BatchNormalization(axis=-1))               
convLayer01 = Activation('relu')                            
model.add(convLayer01)

# CNN Layer 2
model.add(Conv2D(32, (3, 3)))                        
model.add(BatchNormalization(axis=-1))               
model.add(Activation('relu'))                        
convLayer02 = MaxPooling2D(pool_size=(2,2))          
model.add(convLayer02)

# CNN Layer 3
model.add(Conv2D(64,(3, 3)))                         
model.add(BatchNormalization(axis=-1))               
convLayer03 = Activation('relu')                     
model.add(convLayer03)

# CNN Layer 4
model.add(Conv2D(64, (3, 3)))                        
model.add(BatchNormalization(axis=-1))               
model.add(Activation('relu'))                        
convLayer04 = MaxPooling2D(pool_size=(2,2))          
model.add(convLayer04)
model.add(Flatten())                                 

# CNN Layer 5
model.add(Dense(512))                                
model.add(BatchNormalization())                      
model.add(Activation('relu'))                        

# CNN Layer 6                       
model.add(Dropout(0.2))                              
model.add(Dense(10))                                 
model.add(Activation('softmax'))                     

In [ ]:
# we can do another way This CNN by saying kernel size, filter size in Array or list
"""
nn = [16,32,64,128,256]
fs = [(3,3),(5,5),(7,7)]
#ps = [(2,2),(3,3)]
ps = [(2,2)]
conv_layers = [0,2,4,6]
dense_layers = [1,2,3,4,5]
 
for d in dense_layers:
    for c in conv_layers:
        for n in nn:
            for f in fs:
                for p in ps:
 
                    NAME = "{}-conv-{}-neuron-{}-dense-{}-filtersize-{}-pool-size".format(c,n,d,f,p)
                    model = Sequential()
 
                    #model.add( Conv2D(n), f, input_shape=(32,32,3) )
                    model.add(Conv2D(n, f, input_shape=(28,28,1))) 
                    model.add( Activation('relu') )
                    model.add(BatchNormalization(axis=-1)) 
                    model.add(MaxPooling2D(pool_size=(p)))          # Pool the max values over a 2x2 kernel
                    
 
                   
                    for i in range(c-1):  
                        model.add( Conv2D(n, f))
                        model.add(Activation('relu')) 
                        model.add(BatchNormalization(axis=-1)) 
                        model.add(MaxPooling2D(pool_size=(2,2)))
                        model.add( Flatten() )
                    for i in range(d):  
                        model.add( Dense(n) )
                        model.add( Activation('relu') )
                        model.add( Dense(10) )
                        model.add( Activation('softmax') )
                     
model.summary()

""" 

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0

In [7]:
# optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Keras has a great built-in feature to do automatic augmentation
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

In [9]:
# this method actually results in significant memory savings
train_generator = gen.flow(X_train, Y_train, batch_size=128)
test_generator = test_gen.flow(X_test, Y_test, batch_size=128)

In [10]:
# SIGNIFICANT MEMORY SAVINGS (important for larger, deeper networks)

model.fit_generator(train_generator, steps_per_epoch=60000//128, epochs=10, verbose=1, validation_data=test_generator, validation_steps=10000//128)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
468/468 [==============================] - 60s 63ms/step - loss: 0.2798 - accuracy: 0.9110 - val_loss: 0.0786 - val_accuracy: 0.9768
Epoch 2/10
468/468 [==============================] - 29s 61ms/step - loss: 0.0555 - accuracy: 0.9827 - val_loss: 0.0281 - val_accuracy: 0.9912
Epoch 3/10
468/468 [==============================] - 29s 61ms/step - loss: 0.0390 - accuracy: 0.9872 - val_loss: 0.0250 - val_accuracy: 0.9915
Epoch 4/10
468/468 [==============================] - 29s 62ms/step - loss: 0.0338 - accuracy: 0.9891 - val_loss: 0.0326 - val_accuracy: 0.9895
Epoch 5/10
468/468 [==============================] - 29s 62ms/step - loss: 0.0290 - accuracy: 0.9907 - val_loss: 0.0310 - val_accuracy: 0.9895
Epoch 6/10
468/468 [==============================] - 29s 61ms/step - loss: 0.0308 - accuracy: 0.9904 - val_loss: 0.0385 - val_accuracy: 0.9857
Epoch 7/10
468/468 [==============================] - 29s 62ms/step - loss: 0.0237 - accuracy: 0.9920 - val_loss: 0.0276 - val_accuracy:

In [11]:
score = model.evaluate(X_test, Y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 2s 5ms/step - loss: 0.0200 - accuracy: 0.9939
Test score: 0.020008021965622902
Test accuracy: 0.9939000010490417
